In [1]:
import lightgbm
from scipy import ndimage
import pandas as pd

import boto3
import boto
import s3fs
import multiprocessing as mp
import os
import pyarrow.parquet as pq
import random
from sklearn.utils import shuffle
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score

In [2]:
#lambdarank2obj

def train_bst(fn_train, fn_valid, obj1_label='label'):

    train_data = lightgbm.Dataset(fn_train)
    valid_data = lightgbm.Dataset(fn_valid)

    param = {
        'boosting_type':'gbdt',
        "objective": 'lambdarank',
        'metric': 'ndcg',
        'ndcg_eval_at': [1, 3, 5, 10],
        'max_depth':12,
        'random_state':42,
        'num_iterations':200,
        'lambdarank_norm':True,
        'ignore_column':'name:label2',
        'label_column':'name:{}'.format(obj1_label),
        'group_column':'name:query_id',
        'header': 'true'
    }
    
    bst = lightgbm.train(
        param, train_data, 
        valid_sets=[valid_data], valid_names=["valid"],\
        verbose_eval=10)
    
    return bst

def train_bst_two_obj(fn_train, fn_valid, obj1_label='label',
              obj2_label='label2',trunc_obj2=30,weight_obj2=0.3):

    print("weight_obj2",weight_obj2)
    train_data = lightgbm.Dataset(fn_train)
    valid_data = lightgbm.Dataset(fn_valid)

    param = {
        'boosting_type':'gbdt',
        "objective": "lambdarank2obj",
        'metric': 'ndcg',
        'ndcg_eval_at': [1, 3, 5, 10],
        'max_depth':12,
        'random_state':42,
        'num_iterations':200,
        'lambdarank_norm':True,
        'lambdarank_truncation_level_obj2':trunc_obj2,
        'weight_obj2':weight_obj2,
        'label_column':'name:{}'.format(obj1_label),
        'label2_column':'name:{}'.format(obj2_label),
        'group_column':'name:query_id',
        'header': 'true'
    }
    
    bst = lightgbm.train(
        param, train_data, 
        valid_sets=[valid_data], valid_names=["valid"],\
        verbose_eval=10)
    
    return bst

In [3]:
def apply_ranker(fn, fn_out, bst, query_col, score_col):
    df = pd.read_csv(fn)
    df[score_col] = bst.predict(fn, data_has_header=True)
    df.to_csv(fn_out,index=False)
    return

def compute_stats(fn, query_col, score_col, labels=['label','label2'], eval_at=[1,3,5,10]):
    df = pd.read_csv(fn)
    a = []
    grouped = df.groupby(query_col)
    colnames = ["ndcg@{}".format(x) for x in eval_at]
    for c in labels:
        l_ndcg_scores=[]
        for name, group in grouped:
            a1 = group[c].values
            a2 = group['score'].values
            if (len(a1)==1):
                continue
            q = [ndcg_score([a1],[a2], k=x) for x in eval_at]
            l_ndcg_scores.append(q)
        a.append(np.array(l_ndcg_scores).mean(axis=0))
    return pd.DataFrame(np.array(a), columns=colnames)

In [4]:
def run_model(pn):
    fn_train = "./train{}.txt".format(pn)
    fn_valid = "./vali{}.txt".format(pn)
    fn_test = "./test{}.txt".format(pn)
    l_stats = {}
    for w in np.linspace(0.1,0.9,50):
        bst_lamdarank_2obj = train_bst_two_obj(fn_train, fn_valid, obj1_label='label',
                  obj2_label='label2',trunc_obj2=30,weight_obj2=w)
        apply_ranker(fn_test, "./scored_test_data.csv", bst_lamdarank_2obj, 'query_id', 'score')
        stat = compute_stats("./scored_test_data.csv", 'query_id', 'score', 
                          labels=['label','label2'], eval_at=[1,3,5,10])
        stat['w']=w
        l_stats[w]=stat    
    all_stats=pd.concat(l_stats.values())
    return all_stats

In [5]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots



def plot_result(df, pn, y_lim=[0,1]):
    
    scenario_txt="primary vs secondary ndcg@10 ({}% secondary label error)".format(pn)
    
    df = df[df['pn']==pn] 
    ndcg_pr = df[df['objective'] == 'primary']
    ndcg_sc = df[df['objective'] == 'secondary']
    w = ndcg_pr['w']

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Scatter(x=w, y=ndcg_pr['ndcg@10'], 
                             textposition="bottom center",
                        mode='lines+markers+text',
                        name='primary objective ndcg@10'),secondary_y=False)
    fig.add_trace(go.Scatter(x=w, y=ndcg_sc['ndcg@10'], 
                             textposition="bottom center",
                        mode='lines+markers+text',
                        name='secondary objective ndcg@10'),secondary_y=True)
    fig.update_traces(textposition="bottom right")
    fig.update_layout(
        title=scenario_txt,
        title_x=0.5,
        title_xanchor = "right",
        xaxis_title="w",
        yaxis_title="primary objective ndcg@10",
        legend_title="",
        font=dict(
            family="Courier New, monospace",
            size=10,
            color="RebeccaPurple"
        )
    )
    fig.update_yaxes(title_text="secondary objective ndcg@10",range=y_lim, secondary_y=True)
    fig.update_yaxes(range=y_lim, secondary_y=False)

    fig.show()

# Lambdarank two objectives

In [6]:
l_stats = {}
for pn in [10, 30, 50]:
    stat = run_model(pn)
    stat['pn'] = pn
    l_stats[pn] = stat
all_stats=pd.concat(l_stats.values())

weight_obj2 0.1
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Using column label as label
[LightGBM] [Info] Using column label2 as label2
[LightGBM] [Info] Using column query_id as group/query id


/Users/caldanarosenbe/opt/anaconda3/envs/ds2/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/caldanarosenbe/opt/anaconda3/envs/ds2/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Construct bin mappers from text data time 1.04 seconds
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.098655 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 26101
[LightGBM] [Info] Number of data points in the train set: 723412, number of used features: 138
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Using column label as label
[LightGBM] [Info] Using column label2 as label2
[LightGBM] [Info] Using column query_id as group/query id
[10]	valid's ndcg@1: 0.438478	valid's ndcg@3: 0.429945	valid's ndcg@5: 0.433878	valid's ndcg@10: 0.454881
[20]	valid's ndcg@1: 0.458925	valid's ndcg@3: 0.44

/Users/caldanarosenbe/opt/anaconda3/envs/ds2/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/caldanarosenbe/opt/anaconda3/envs/ds2/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Construct bin mappers from text data time 0.94 seconds
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.117818 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 26101
[LightGBM] [Info] Number of data points in the train set: 723412, number of used features: 138
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Using column label as label
[LightGBM] [Info] Using column label2 as label2
[LightGBM] [Info] Using column query_id as group/query id
[10]	valid's ndcg@1: 0.444175	valid's ndcg@3: 0.432849	valid's ndcg@5: 0.439792	valid's ndcg@10: 0.45788
[20]	valid's ndcg@1: 0.460795	valid's ndcg@3: 0.445

...

[LightGBM] [Info] Construct bin mappers from text data time 0.93 seconds
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.087886 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 26101
[LightGBM] [Info] Number of data points in the train set: 723412, number of used features: 138
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Using column label as label
[LightGBM] [Info] Using column label2 as label2
[LightGBM] [Info] Using column query_id as group/query id
[10]	valid's ndcg@1: 0.201498	valid's ndcg@3: 0.20706	valid's ndcg@5: 0.218777	valid's ndcg@10: 0.256928
[20]	valid's ndcg@1: 0.242589	valid's ndcg@3: 0.236

/Users/caldanarosenbe/opt/anaconda3/envs/ds2/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/caldanarosenbe/opt/anaconda3/envs/ds2/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Construct bin mappers from text data time 0.96 seconds
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.078664 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 26101
[LightGBM] [Info] Number of data points in the train set: 723412, number of used features: 138
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Using column label as label
[LightGBM] [Info] Using column label2 as label2
[LightGBM] [Info] Using column query_id as group/query id
[10]	valid's ndcg@1: 0.180756	valid's ndcg@3: 0.190792	valid's ndcg@5: 0.208262	valid's ndcg@10: 0.243694
[20]	valid's ndcg@1: 0.219321	valid's ndcg@3: 0.21

/Users/caldanarosenbe/opt/anaconda3/envs/ds2/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/caldanarosenbe/opt/anaconda3/envs/ds2/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Construct bin mappers from text data time 0.91 seconds
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.079759 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 26101
[LightGBM] [Info] Number of data points in the train set: 723412, number of used features: 138
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Using column label as label
[LightGBM] [Info] Using column label2 as label2
[LightGBM] [Info] Using column query_id as group/query id
[10]	valid's ndcg@1: 0.158373	valid's ndcg@3: 0.177252	valid's ndcg@5: 0.196897	valid's ndcg@10: 0.23162
[20]	valid's ndcg@1: 0.193647	valid's ndcg@3: 0.204

In [6]:
all_stats = pd.read_csv("./performance.csv")

In [7]:
all_stats.columns = ['objective'] + list(all_stats.columns[1:])

In [8]:
all_stats

,objective,ndcg@1,ndcg@3,ndcg@5,ndcg@10,w,pn
0,primary,0.565826,0.561370,0.558435,0.559827,0.100000,10
1,secondary,0.478858,0.468239,0.467878,0.472818,0.100000,10
2,primary,0.571128,0.559378,0.558912,0.560675,0.116327,10
3,secondary,0.481593,0.470650,0.467791,0.474149,0.116327,10
4,primary,0.570728,0.562048,0.558931,0.560745,0.132653,10
...,...,...,...,...,...,...,...
295,secondary,0.654262,0.649037,0.638322,0.619492,0.867347,50
296,primary,0.433273,0.413806,0.411978,0.424466,0.883673,50
297,secondary,0.679872,0.663389,0.655316,0.633117,0.883673,50
298,primary,0.397359,0.384124,0.390238,0.402866,0.900000,50


In [10]:
plot_result(all_stats,10)